# Basics methods for training and prediction using Keras

In [1]:
from IPython.display import display, Markdown
with open('setup.md', 'r') as fh:
    content = fh.read()
display(Markdown(content))

Define the following env variable before starting Jupyter Lab:  
`export DIR_PROJ=your_path_git_repository`  
`export PYTHONPATH=$DIR_PROJ`  
`export PATH_TENSORBOARD=your_path_tensorboard`  
`export PATH_DATASETS=your_path_datasets`  
`export PROJECT_ID=your_gcp_project_id`  
`export BUCKET_NAME=your_gcp_gs_bucket_name`  
`export REGION=you_region`  
  
Start Jupyter Lab:  
`jupyter lab`  
  
Choose the proper Anaconda python environment:  
`Python [conda env:env_tensorflow]`  
    

## Import packages

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from absl import logging
import os

## Define paths

In [3]:
try:
    data_dir=os.environ['PATH_DATASETS']
except:
    print('missing PATH_DATASETS')

## Getting and preprocessing data

In [4]:
logging.set_verbosity(logging.INFO)
# Define the estimator's input_fn
STEPS_PER_EPOCH = 5
BUFFER_SIZE = 10 # Use a much larger value for real code. 
BATCH_SIZE = 64
NUM_EPOCHS = 5

In [5]:
def input_fn():
    datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True, data_dir=data_dir)
    mnist_train, mnist_test = datasets['train'], datasets['test']

    BUFFER_SIZE = 10000
    BATCH_SIZE = 64

    def scale(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255
    
        return image, label[..., tf.newaxis]

    train_data = mnist_train.repeat().map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    test_data = mnist_test.repeat().map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    return train_data, test_data

In [6]:
train_data, test_data=input_fn()

INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Field info.location from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset mnist (/Users/tarrade/tensorflow_datasets/mnist/3.0.0)
INFO:absl:Constructing tf.data.Dataset for split None, from /Users/tarrade/tensorflow_datasets/mnist/3.0.0


## Build the model

In [7]:
def make_model():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu',
                               kernel_regularizer=tf.keras.regularizers.l2(0.02),
                               input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

In [8]:
model = make_model()

In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 5408)              0         
_________________________________________________________________
dropout (Dropout)            (None, 5408)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                346176    
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                6

## Training the model

In [11]:
print("train")
model.fit(train_data,
          steps_per_epoch=5,
          epochs=10,
          verbose = 1)

train
Train for 5 steps
Epoch 1/10
5/5 [==============================] - 2s 316ms/step - loss: 1.7140 - accuracy: 0.4688
Epoch 2/10
5/5 [==============================] - 0s 12ms/step - loss: 0.8979 - accuracy: 0.7719
Epoch 3/10
5/5 [==============================] - 0s 13ms/step - loss: 0.6559 - accuracy: 0.8438
Epoch 4/10
5/5 [==============================] - 0s 13ms/step - loss: 0.5751 - accuracy: 0.8594
Epoch 5/10
5/5 [==============================] - 0s 14ms/step - loss: 0.4160 - accuracy: 0.9219
Epoch 6/10
5/5 [==============================] - 0s 13ms/step - loss: 0.4639 - accuracy: 0.9000
Epoch 7/10
5/5 [==============================] - 0s 15ms/step - loss: 0.3683 - accuracy: 0.9094
Epoch 8/10
5/5 [==============================] - 0s 14ms/step - loss: 0.3386 - accuracy: 0.9219
Epoch 9/10
5/5 [==============================] - 0s 14ms/step - loss: 0.3841 - accuracy: 0.8938
Epoch 10/10
5/5 [==============================] - 0s 14ms/step - loss: 0.3324 - accuracy: 0.9375


## Evaluating the model

In [12]:
print("evaluate on training dataset")
model.evaluate(train_data,
              steps=1)

evaluate on training dataset
1/1 [==============================] - 1s 931ms/step - loss: 1.4045 - accuracy: 0.8594


[1.4045376777648926, 0.859375]

In [13]:
print("evaluate on training dataset")
model.evaluate(test_data,
               steps=1)

evaluate on training dataset
1/1 [==============================] - 1s 844ms/step - loss: 1.4343 - accuracy: 0.8281


[1.4342684745788574, 0.828125]

## Doing prediction withn our model

In [14]:
print("predict on one batch of data")
print(model.predict_on_batch(train_data))

predict on one batch of data
tf.Tensor(
[[0.0736251  0.05164756 0.07825514 0.09896357 0.08101612 0.04053068
  0.0887577  0.21287563 0.07838576 0.19594276]
 [0.08664329 0.06998219 0.05613408 0.08766057 0.05766976 0.14650126
  0.08477965 0.10850022 0.18344976 0.11867922]
 [0.14416239 0.04268898 0.08640311 0.11793164 0.06320393 0.06087663
  0.101133   0.05855999 0.21505839 0.10998195]
 [0.04819774 0.17785509 0.1688584  0.08626056 0.08241254 0.04044393
  0.07360231 0.06877417 0.17206435 0.08153084]
 [0.0484409  0.07036085 0.03904511 0.12578669 0.06098217 0.22633488
  0.08878732 0.09147447 0.1471274  0.10166022]
 [0.09628263 0.06511059 0.12121519 0.16074607 0.05405232 0.04132611
  0.21123596 0.07676069 0.11540928 0.05786118]
 [0.27816433 0.05708036 0.076178   0.07828818 0.06528085 0.05823449
  0.14520192 0.06844957 0.09210934 0.08101302]
 [0.06352511 0.07114305 0.07539798 0.12155282 0.08518842 0.04471709
  0.08011513 0.20677991 0.11683664 0.13474388]
 [0.04642796 0.0757155  0.08461747 0.073

In [15]:
print("test on one batch of data")
print(model.test_on_batch(train_data))

test on one batch of data
[1.5167072, 0.8046875]
